# Data Consolidation
#### Building a Dataset 

In [1]:
# library load
import pandas as pd
import numpy as np
%matplotlib inline

In [14]:
elim_data = pd.read_csv('bachelorette_538.csv')
# remove first line, which is just the header info repeated
elim_data = elim_data.drop([0])
elim_data.head()

,SHOW,SEASON,CONTESTANT,ELIMINATION-1,ELIMINATION-2,ELIMINATION-3,ELIMINATION-4,ELIMINATION-5,ELIMINATION-6,ELIMINATION-7,...,DATES-1,DATES-2,DATES-3,DATES-4,DATES-5,DATES-6,DATES-7,DATES-8,DATES-9,DATES-10
1,Bachelorette,13,13_BRYAN_A,R1,NaN,NaN,R,R,NaN,R,...,NaN,NaN,D6,D13,D1,D7,D1,D1,D1,D1
2,Bachelorette,13,13_PETER_K,NaN,R,NaN,NaN,NaN,R,R,...,NaN,D1,D6,D13,D9,D7,D1,D1,D1,D1
3,Bachelorette,13,13_ERIC_B,NaN,NaN,R,NaN,NaN,R,R,...,NaN,D10,D8,D13,D9,D1,D3,D1,D1,NaN
4,Bachelorette,13,13_DEAN_U,NaN,R,NaN,R,NaN,NaN,R,...,NaN,D8,D8,D1,D9,D7,D1,D1,NaN,NaN
5,Bachelorette,13,13_ADAM_G,NaN,NaN,NaN,NaN,NaN,NaN,ED,...,NaN,D10,D8,D13,D9,D7,D3,NaN,NaN,NaN


In [16]:
# need to change from wide form data to long form data
# want it to be:
# SHOW, SEASON, CONTESTANT, 'TYPE', 'VALUE'
# with 'TYPE' as the current column names
elim1 = pd.melt(elim_data, id_vars=['SHOW', 'SEASON', 'CONTESTANT'])
elim1.head()

,SHOW,SEASON,CONTESTANT,variable,value
0,Bachelorette,13,13_BRYAN_A,ELIMINATION-1,R1
1,Bachelorette,13,13_PETER_K,ELIMINATION-1,NaN
2,Bachelorette,13,13_ERIC_B,ELIMINATION-1,NaN
3,Bachelorette,13,13_DEAN_U,ELIMINATION-1,NaN
4,Bachelorette,13,13_ADAM_G,ELIMINATION-1,NaN


In [17]:
# rename variable and value
elim1.columns = ['SHOW', 'SEASON', 'CONTESTANT', 'TYPE', 'VALUE']
elim1.head()

,SHOW,SEASON,CONTESTANT,TYPE,VALUE
0,Bachelorette,13,13_BRYAN_A,ELIMINATION-1,R1
1,Bachelorette,13,13_PETER_K,ELIMINATION-1,NaN
2,Bachelorette,13,13_ERIC_B,ELIMINATION-1,NaN
3,Bachelorette,13,13_DEAN_U,ELIMINATION-1,NaN
4,Bachelorette,13,13_ADAM_G,ELIMINATION-1,NaN


In [18]:
# split TYPE variable
elim1[['ELIM_DATE', 'NUMBER']] = elim1['TYPE'].str.split('-',expand=True)

In [19]:
elim1.head()

,SHOW,SEASON,CONTESTANT,TYPE,VALUE,ELIM_DATE,NUMBER
0,Bachelorette,13,13_BRYAN_A,ELIMINATION-1,R1,ELIMINATION,1
1,Bachelorette,13,13_PETER_K,ELIMINATION-1,NaN,ELIMINATION,1
2,Bachelorette,13,13_ERIC_B,ELIMINATION-1,NaN,ELIMINATION,1
3,Bachelorette,13,13_DEAN_U,ELIMINATION-1,NaN,ELIMINATION,1
4,Bachelorette,13,13_ADAM_G,ELIMINATION-1,NaN,ELIMINATION,1


In [20]:
elim1.to_csv("elimination_long_form.csv")

### this is the information we need to model 'survival'

# Need to Bring In the Contestant & Bachelor/ette data
#### modify region/state and occupation
### Then combine it to create a possible demographic predictive set
#### and make some other possible useful variables

In [2]:
bach = pd.read_csv("bachelors.csv")
bach_cont = pd.read_csv("bachelor-contestants.csv")
ette = pd.read_csv("bachelorettes.csv")
ette_cont = pd.read_csv("bachelorette-contestants.csv")

In [3]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

### Bachelors First

In [9]:
bach.head()

,Name,Age,Hometown,Height,Season
0,Alex Michel,32,"Charlottesville, Virginia",NaN,1
1,Aaron Buerge,28,"Butler, Missouri",NaN,2
2,Jesse Palmer,34,"Toronto, Ontario",NaN,5
3,Lorenzo Borghese,34,"Milan, Italy",NaN,9
4,Andy Baldwin,30,"Lancaster, Pennsylvania",NaN,10


In [10]:
# Split Hometown column
bach[['Hometown_City', 'Hometown_State']] = bach['Hometown'].str.split(', ',expand=True)
bach.head()

,Name,Age,Hometown,Height,Season,Hometown_City,Hometown_State
0,Alex Michel,32,"Charlottesville, Virginia",NaN,1,Charlottesville,Virginia
1,Aaron Buerge,28,"Butler, Missouri",NaN,2,Butler,Missouri
2,Jesse Palmer,34,"Toronto, Ontario",NaN,5,Toronto,Ontario
3,Lorenzo Borghese,34,"Milan, Italy",NaN,9,Milan,Italy
4,Andy Baldwin,30,"Lancaster, Pennsylvania",NaN,10,Lancaster,Pennsylvania


In [11]:
# Convert Hometown_State to us state abbreviation if relevant
StateAbbrev = []
for index, row in bach.iterrows():
    #print(row['c1'], row['c2'])
    if row['Hometown_State'] in us_state_abbrev: 
        StateAbbrev.append(us_state_abbrev[row['Hometown_State']])
    else: 
        StateAbbrev.append(row['Hometown_State'])
        
bach['Hometown_StateAbbrev'] = StateAbbrev
bach.head()

,Name,Age,Hometown,Height,Season,Hometown_City,Hometown_State,Hometown_StateAbbrev
0,Alex Michel,32,"Charlottesville, Virginia",NaN,1,Charlottesville,Virginia,VA
1,Aaron Buerge,28,"Butler, Missouri",NaN,2,Butler,Missouri,MO
2,Jesse Palmer,34,"Toronto, Ontario",NaN,5,Toronto,Ontario,Ontario
3,Lorenzo Borghese,34,"Milan, Italy",NaN,9,Milan,Italy,Italy
4,Andy Baldwin,30,"Lancaster, Pennsylvania",NaN,10,Lancaster,Pennsylvania,PA


### Variables We'll Keep:
#### Name
#### Age
#### Height
#### Season
#### Hometown_City
#### Hometown_StateAbbrev

###### Removing the summary Hometown column (we've broken it up), Hometown_State (because we're using the abbreviation version instead)

In [12]:
final_bach = bach[['Name', 'Age', 'Height', 'Season', 'Hometown_City', 'Hometown_StateAbbrev']].copy()
final_bach.columns = ['BACHELOR', 'BACHELOR_AGE', 'BACHELOR_HEIGHT', 'SEASON', 'BACHELOR_HOMETOWN_CITY', 'BACHELOR_HOMETOWN_STATE']
final_bach.head()

,BACHELOR,BACHELOR_AGE,BACHELOR_HEIGHT,SEASON,BACHELOR_HOMETOWN_CITY,BACHELOR_HOMETOWN_STATE
0,Alex Michel,32,NaN,1,Charlottesville,VA
1,Aaron Buerge,28,NaN,2,Butler,MO
2,Jesse Palmer,34,NaN,5,Toronto,Ontario
3,Lorenzo Borghese,34,NaN,9,Milan,Italy
4,Andy Baldwin,30,NaN,10,Lancaster,PA


### Bachelor Contestants

In [13]:
bach_cont.head()

,Name,Age,Occupation,Hometown,Height,ElimWeek,Season
0,Amanda Marsh,23.0,Event Planner,"Chanute, Kansas",NaN,NaN,1
1,Trista Rehn,29.0,Miami Heat Dancer,"Miami, Florida",NaN,6.0,1
2,Shannon Oliver,24.0,Financial Management Consultant,"Dallas, Texas",NaN,5.0,1
3,Kim,24.0,Nanny,"Tempe, Arizona",NaN,4.0,1
4,Cathy Grimes,22.0,Graduate Student,"Terra Haute, Indiana",NaN,3.0,1


In [14]:
bach_cont['Height'].value_counts()

67.00    6
64.00    5
65.00    3
62.00    3
66.00    3
68.00    2
61.00    2
67.75    1
63.50    1
63.00    1
62.50    1
70.00    1
67.50    1
Name: Height, dtype: int64

In [16]:
# Split Hometown column
bach_cont[['Hometown_City', 'Hometown_State', 'Other']] = bach_cont['Hometown'].str.split(', ',expand=True)
bach_cont.head()

,Name,Age,Occupation,Hometown,Height,ElimWeek,Season,Hometown_City,Hometown_State,Other
0,Amanda Marsh,23.0,Event Planner,"Chanute, Kansas",NaN,NaN,1,Chanute,Kansas,None
1,Trista Rehn,29.0,Miami Heat Dancer,"Miami, Florida",NaN,6.0,1,Miami,Florida,None
2,Shannon Oliver,24.0,Financial Management Consultant,"Dallas, Texas",NaN,5.0,1,Dallas,Texas,None
3,Kim,24.0,Nanny,"Tempe, Arizona",NaN,4.0,1,Tempe,Arizona,None
4,Cathy Grimes,22.0,Graduate Student,"Terra Haute, Indiana",NaN,3.0,1,Terra Haute,Indiana,None


In [17]:
bach_cont['Other'].value_counts()
# we can just drop this

Canada    2
Name: Other, dtype: int64

In [18]:
# Convert Hometown_State to us state abbreviation if relevant
StateAbbrev = []
for index, row in bach_cont.iterrows():
    #print(row['c1'], row['c2'])
    if row['Hometown_State'] in us_state_abbrev: 
        StateAbbrev.append(us_state_abbrev[row['Hometown_State']])
    else: 
        StateAbbrev.append(row['Hometown_State'])
        
bach_cont['Hometown_StateAbbrev'] = StateAbbrev
bach_cont.head()

,Name,Age,Occupation,Hometown,Height,ElimWeek,Season,Hometown_City,Hometown_State,Other,Hometown_StateAbbrev
0,Amanda Marsh,23.0,Event Planner,"Chanute, Kansas",NaN,NaN,1,Chanute,Kansas,None,KS
1,Trista Rehn,29.0,Miami Heat Dancer,"Miami, Florida",NaN,6.0,1,Miami,Florida,None,FL
2,Shannon Oliver,24.0,Financial Management Consultant,"Dallas, Texas",NaN,5.0,1,Dallas,Texas,None,TX
3,Kim,24.0,Nanny,"Tempe, Arizona",NaN,4.0,1,Tempe,Arizona,None,AZ
4,Cathy Grimes,22.0,Graduate Student,"Terra Haute, Indiana",NaN,3.0,1,Terra Haute,Indiana,None,IN


In [19]:
final_cont = bach_cont[['Name', 'Age', 'Occupation', 'Height', 'Season', 'ElimWeek', 'Hometown_City', 'Hometown_StateAbbrev']].copy()
final_cont.columns = ['CONTESTANT', 'CONTESTANT_AGE', 'CONTESTANT_JOB', 'CONTESTANT_HEIGHT', 'SEASON', 'CONTESTANT_ELIMWEEK', 'CONTESTANT_HOMETOWN_CITY', 'CONTESTANT_HOMETOWN_STATE']
final_cont.head()

,CONTESTANT,CONTESTANT_AGE,CONTESTANT_JOB,CONTESTANT_HEIGHT,SEASON,CONTESTANT_ELIMWEEK,CONTESTANT_HOMETOWN_CITY,CONTESTANT_HOMETOWN_STATE
0,Amanda Marsh,23.0,Event Planner,NaN,1,NaN,Chanute,KS
1,Trista Rehn,29.0,Miami Heat Dancer,NaN,1,6.0,Miami,FL
2,Shannon Oliver,24.0,Financial Management Consultant,NaN,1,5.0,Dallas,TX
3,Kim,24.0,Nanny,NaN,1,4.0,Tempe,AZ
4,Cathy Grimes,22.0,Graduate Student,NaN,1,3.0,Terra Haute,IN


##### Merge final_bach and final_cont on SEASON

In [20]:
the_bachelor = final_bach.merge(final_cont, on='SEASON')
print(the_bachelor.shape)
the_bachelor.head()

(423, 13)


,BACHELOR,BACHELOR_AGE,BACHELOR_HEIGHT,SEASON,BACHELOR_HOMETOWN_CITY,BACHELOR_HOMETOWN_STATE,CONTESTANT,CONTESTANT_AGE,CONTESTANT_JOB,CONTESTANT_HEIGHT,CONTESTANT_ELIMWEEK,CONTESTANT_HOMETOWN_CITY,CONTESTANT_HOMETOWN_STATE
0,Alex Michel,32,NaN,1,Charlottesville,VA,Amanda Marsh,23.0,Event Planner,NaN,NaN,Chanute,KS
1,Alex Michel,32,NaN,1,Charlottesville,VA,Trista Rehn,29.0,Miami Heat Dancer,NaN,6.0,Miami,FL
2,Alex Michel,32,NaN,1,Charlottesville,VA,Shannon Oliver,24.0,Financial Management Consultant,NaN,5.0,Dallas,TX
3,Alex Michel,32,NaN,1,Charlottesville,VA,Kim,24.0,Nanny,NaN,4.0,Tempe,AZ
4,Alex Michel,32,NaN,1,Charlottesville,VA,Cathy Grimes,22.0,Graduate Student,NaN,3.0,Terra Haute,IN


In [21]:
the_bachelor.to_csv("the_bachelor.csv")

### Bachelorettes

In [4]:
ette.head()

,Name,Age,Occupation,Hometown,Season
0,Trista Rehn,30.0,Physical therapist,"Indianapolis, Indiana",1
1,Meredith Phillips,NaN,Makeup artist,NaN,2
2,"Jennifer ""Jen"" Schefft",NaN,Publicist,NaN,3
3,DeAnna Pappas,NaN,Real estate agent,NaN,4
4,Jillian Harris,29.0,Interior designer,"Vancouver, British Columbia, Canada",5


In [5]:
# Split Hometown column
ette[['Hometown_City', 'Hometown_State', 'Other']] = ette['Hometown'].str.split(', ',expand=True)
ette.head()
# can remove Other column

,Name,Age,Occupation,Hometown,Season,Hometown_City,Hometown_State,Other
0,Trista Rehn,30.0,Physical therapist,"Indianapolis, Indiana",1,Indianapolis,Indiana,None
1,Meredith Phillips,NaN,Makeup artist,NaN,2,NaN,NaN,NaN
2,"Jennifer ""Jen"" Schefft",NaN,Publicist,NaN,3,NaN,NaN,NaN
3,DeAnna Pappas,NaN,Real estate agent,NaN,4,NaN,NaN,NaN
4,Jillian Harris,29.0,Interior designer,"Vancouver, British Columbia, Canada",5,Vancouver,British Columbia,Canada


In [6]:
# Convert Hometown_State to us state abbreviation if relevant
StateAbbrev = []
for index, row in ette.iterrows():
    #print(row['c1'], row['c2'])
    if row['Hometown_State'] in us_state_abbrev: 
        StateAbbrev.append(us_state_abbrev[row['Hometown_State']])
    else: 
        StateAbbrev.append(row['Hometown_State'])
        
ette['Hometown_StateAbbrev'] = StateAbbrev
ette.head()

,Name,Age,Occupation,Hometown,Season,Hometown_City,Hometown_State,Other,Hometown_StateAbbrev
0,Trista Rehn,30.0,Physical therapist,"Indianapolis, Indiana",1,Indianapolis,Indiana,None,IN
1,Meredith Phillips,NaN,Makeup artist,NaN,2,NaN,NaN,NaN,NaN
2,"Jennifer ""Jen"" Schefft",NaN,Publicist,NaN,3,NaN,NaN,NaN,NaN
3,DeAnna Pappas,NaN,Real estate agent,NaN,4,NaN,NaN,NaN,NaN
4,Jillian Harris,29.0,Interior designer,"Vancouver, British Columbia, Canada",5,Vancouver,British Columbia,Canada,British Columbia


In [7]:
final_ette = ette[['Name', 'Age', 'Occupation', 'Season', 'Hometown_City', 'Hometown_StateAbbrev']].copy()
final_ette.columns = ['BACHELORETTE', 'BACHELORETTE_AGE', 'BACHELORETTE_JOB', 'SEASON', 'BACHELORETTE_HOMETOWN_CITY', 'BACHELORETTE_HOMETOWN_STATE']
final_ette.head()

,BACHELORETTE,BACHELORETTE_AGE,BACHELORETTE_JOB,SEASON,BACHELORETTE_HOMETOWN_CITY,BACHELORETTE_HOMETOWN_STATE
0,Trista Rehn,30.0,Physical therapist,1,Indianapolis,IN
1,Meredith Phillips,NaN,Makeup artist,2,NaN,NaN
2,"Jennifer ""Jen"" Schefft",NaN,Publicist,3,NaN,NaN
3,DeAnna Pappas,NaN,Real estate agent,4,NaN,NaN
4,Jillian Harris,29.0,Interior designer,5,Vancouver,British Columbia


### Bachelorette Contestants

In [8]:
ette_cont.head()

,Name,Age,Occupation,Hometown,ElimWeek,Season
0,Ryan Sutter,29,Firefighter,"Vail, Colorado",NaN,1
1,Charlie Maher,28,Financial Analyst,"Hermosa Beach, California",6.0,1
2,Russ,30,Writer,"San Rafael, California",5.0,1
3,Greg T.,28,Importer,"Manhattan, New York",4.0,1
4,Bob Guiney,31,Mortgage Broker,"Ferndale, Michigan",3.0,1


In [9]:
# Split Hometown column
ette_cont[['Hometown_City', 'Hometown_State']] = ette_cont['Hometown'].str.split(', ',expand=True)
ette_cont.head()
# can remove Other column

,Name,Age,Occupation,Hometown,ElimWeek,Season,Hometown_City,Hometown_State
0,Ryan Sutter,29,Firefighter,"Vail, Colorado",NaN,1,Vail,Colorado
1,Charlie Maher,28,Financial Analyst,"Hermosa Beach, California",6.0,1,Hermosa Beach,California
2,Russ,30,Writer,"San Rafael, California",5.0,1,San Rafael,California
3,Greg T.,28,Importer,"Manhattan, New York",4.0,1,Manhattan,New York
4,Bob Guiney,31,Mortgage Broker,"Ferndale, Michigan",3.0,1,Ferndale,Michigan


In [10]:
# Convert Hometown_State to us state abbreviation if relevant
StateAbbrev = []
for index, row in ette_cont.iterrows():
    #print(row['c1'], row['c2'])
    if row['Hometown_State'] in us_state_abbrev: 
        StateAbbrev.append(us_state_abbrev[row['Hometown_State']])
    else: 
        StateAbbrev.append(row['Hometown_State'])
        
ette_cont['Hometown_StateAbbrev'] = StateAbbrev
ette_cont.head()

,Name,Age,Occupation,Hometown,ElimWeek,Season,Hometown_City,Hometown_State,Hometown_StateAbbrev
0,Ryan Sutter,29,Firefighter,"Vail, Colorado",NaN,1,Vail,Colorado,CO
1,Charlie Maher,28,Financial Analyst,"Hermosa Beach, California",6.0,1,Hermosa Beach,California,CA
2,Russ,30,Writer,"San Rafael, California",5.0,1,San Rafael,California,CA
3,Greg T.,28,Importer,"Manhattan, New York",4.0,1,Manhattan,New York,NY
4,Bob Guiney,31,Mortgage Broker,"Ferndale, Michigan",3.0,1,Ferndale,Michigan,MI


In [11]:
final_ette_cont = ette_cont[['Name', 'Age', 'Occupation', 'ElimWeek', 'Season', 'Hometown_City', 'Hometown_StateAbbrev']].copy()
final_ette_cont.columns = ['CONTESTANT', 'CONTESTANT_AGE', 'CONTESTANT_JOB', 'CONTESTANT_ELIMWEEK', 'SEASON', 'CONTESTANT_HOMETOWN_CITY', 'CONTESTANT_HOMETOWN_STATE']
final_ette_cont.head()

,CONTESTANT,CONTESTANT_AGE,CONTESTANT_JOB,CONTESTANT_ELIMWEEK,SEASON,CONTESTANT_HOMETOWN_CITY,CONTESTANT_HOMETOWN_STATE
0,Ryan Sutter,29,Firefighter,NaN,1,Vail,CO
1,Charlie Maher,28,Financial Analyst,6.0,1,Hermosa Beach,CA
2,Russ,30,Writer,5.0,1,San Rafael,CA
3,Greg T.,28,Importer,4.0,1,Manhattan,NY
4,Bob Guiney,31,Mortgage Broker,3.0,1,Ferndale,MI


#### Merge final_ette and final_ette_cont on SEASON

In [12]:
the_bachelorette = final_ette.merge(final_ette_cont, on='SEASON')
print(the_bachelorette.shape)
the_bachelorette.head()

(282, 12)


,BACHELORETTE,BACHELORETTE_AGE,BACHELORETTE_JOB,SEASON,BACHELORETTE_HOMETOWN_CITY,BACHELORETTE_HOMETOWN_STATE,CONTESTANT,CONTESTANT_AGE,CONTESTANT_JOB,CONTESTANT_ELIMWEEK,CONTESTANT_HOMETOWN_CITY,CONTESTANT_HOMETOWN_STATE
0,Trista Rehn,30.0,Physical therapist,1,Indianapolis,IN,Ryan Sutter,29,Firefighter,NaN,Vail,CO
1,Trista Rehn,30.0,Physical therapist,1,Indianapolis,IN,Charlie Maher,28,Financial Analyst,6.0,Hermosa Beach,CA
2,Trista Rehn,30.0,Physical therapist,1,Indianapolis,IN,Russ,30,Writer,5.0,San Rafael,CA
3,Trista Rehn,30.0,Physical therapist,1,Indianapolis,IN,Greg T.,28,Importer,4.0,Manhattan,NY
4,Trista Rehn,30.0,Physical therapist,1,Indianapolis,IN,Bob Guiney,31,Mortgage Broker,3.0,Ferndale,MI


In [13]:
the_bachelorette.to_csv("the_bachelorette.csv")